In [1]:
import numpy as np
import scipy as sp 
import scipy.linalg

## Generating A from 1905.10876 

We genearte $A$ and find $\mid \mid {A} \mid \mid^2_{HS}$

In [2]:
def random_unitary(N):
    """Generate uniformly random unitary from U(N). 
       Credit to Jarrod McClean"""
    Z = np.random.randn(N, N) + 1.0j * np.random.randn(N, N)
    [Q, R] = sp.linalg.qr(Z)
    D = np.diag(np.diagonal(R) / np.abs(np.diagonal(R)))
    return np.dot(Q, D)

We can approximate the integral over $U(N)$ by 
$$ \int_{U(N)} f(U) dU \approx \frac{1}{M} \sum_{i=1}^M f(U_i) $$
where $M$ is the number of samples nd $U_i$ is a uniformly random unitary. 

We can define $f(U) = U |0><0| U^{\dagger}$. 

In [3]:
# Expressibility of 1-qubit Idle circuit consisting of only |0> 
M = 1000 
int_harr = np.zeros((2,2))
start_state = np.array([[1, 0], [0, 0]], dtype='complex128')
for i in range(M):
    u_i = random_unitary(2)
    int_harr = int_harr + (u_i * start_state * np.conjugate(np.transpose(u_i)))
int_harr /= M 

In [4]:
A = int_harr - start_state 
print(np.trace(np.conjugate(np.transpose(A)) * A))

(0.2255817748232015+0j)


In [5]:
# Let's try with |0> --- H --- R_z(a) --- R_X(b)
H = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]], dtype='complex128')
RZ = lambda a: np.array([[np.exp(-1j * a / 2), 0], [0, np.exp(1j * a / 2)]], dtype='complex128')
RX = lambda b: np.array([[np.cos(b / 2), -1j * np.sin(b / 2)], 
                         [-1j * np.sin(b / 2), np.cos(b / 2)]], dtype='complex128')

int_ansatz = np.zeros((2,2))
for i in range(M):
    a = np.random.uniform(-np.pi, np.pi)
    b = np.random.uniform(-np.pi, np.pi)
    
    rho = H * start_state * H 
    rho = RZ(a) * rho * np.conjugate(np.transpose(RZ(a))) 
    rho = RX(b) * rho * np.conjugate(np.transpose(RX(b)))
    int_ansatz = int_ansatz + rho 

int_ansatz /= M

In [6]:
A = int_harr - int_ansatz 
print(np.trace(np.conjugate(np.transpose(A)) * A))

(0.07285261876527102+0j)


Hm okay so expressibility is sort of expressed by how small $\mathrm{Tr} A^\dagger A$ is. 

What if I just defined expressibility as 